In [1]:
import os
from math import log as log
import pandas as pd
import csv
import time
import numpy as np
import networkx as nx

In [2]:
cwd = os.getcwd()

# Getting the Orbit Degree Matrices from the file directory

In [3]:
def make_Orbit_Degree_Matrix_Dictionary(inpath):
    
    ''' Function Purpose: Creates a dictionary called the 'orbit_degree_matrices_dictionary'  whose keys are 
    'days' and values are the pair of file paths for the 'signature' file and node 'dictionary' file. 
    
    Functions Called:
    A. appendPath'''
    
    orbit_degree_matrices_dictionary = {}
    days = []
    
    for txt in os.listdir(inpath):
        # populate the graphlet_vector_matrix_dictionary with paths for the signatures and dictionaries
        
        ii = None
        
        if txt.endswith(".signatures.txt"):
            ii = 0
        elif txt.endswith(".dictionary.txt"):
            ii = 1
        else:
            continue
            
        orbit_degree_matrices_dictionary, days = appendPath(orbit_degree_matrices_dictionary, days, txt, inpath, ii)
    
    #graphlet_vector_matrix_dictionary = dict(sorted(graphlet_vector_matrix_dictionary.items(), key=lambda x: x[0]))

    return orbit_degree_matrices_dictionary, days

In [4]:
def appendPath(orbit_degree_matrices_dictionary, days, txt, inpath, ii):
    ''' This appends the file path to either the signatures set (ie graphlet_degree_vectors) 
    or the the dictionary set'''
    
    day = int(txt.split('.')[0].split('_')[1].split('y')[1])
    
    if not day in days:
        
        days.append(day)
        
        days = sorted(days) # 'days' stores integer values in increasing sequence
        
        orbit_degree_matrices_dictionary[day] = [None, None]
    
    path = inpath + txt
    
    orbit_degree_matrices_dictionary[day][ii] = path
    
    return orbit_degree_matrices_dictionary, days
    

# Getting two Orbit Degree Matrices from a single Well

When we want to store the orbit degree vectors and their node labels (ie their orbit_degree_matrices_dictionary) for two graphs. The time points can be taken consecutively, or even multiple days apart. This length will be called "gap".


In [5]:
def store_Data_For_TimeGraphs(orbit_degree_matrices_dictionary, current_day, next_day, d, gap, orbit_degree_matrix_1, orbit_degree_matrix_2, node_labels_1, node_labels_2):
    
    ''' Function Purpose: To extract the signature files of two graphs with time points.
    
        Variables: 
        1. graphlet_vector_matrix_dictionary := a dictionary which holds information about each node's orbit degree vector with the corresponding node label stored in the dictionary. 
        2. days := this is an ordered array with all the time stamps (unit: day) 
        3. d    := the 'days' array index. 
        4. gap  := this value tells us how far apart (units: day) to sample the graphs. Ex: 0 means adjacent time stamps.
        5. orbit_degree_matrix_1   := for graph #1, this is its orbit degree matrix (whose rows correspond to a specific node and columns are the orbits). 
        6. orbit_degree_matrix_2   := for graph #2, this is its orbit degree matrix (...)
        7. node_labels_1 := for graph #1, this dictionary stores the node label for each of its rows. (Ie. The first row does not necessarily mean node 1)
        8. node_labels_2 := for graph #2, this dictionary stores its node labels. 
        
        Functions Called:
        A. read_OrbitDegree_Matrix
        B. read_Node_Labels           '''

    current_graph = orbit_degree_matrices_dictionary[current_day]
    next_graph = orbit_degree_matrices_dictionary[next_day]


    if gap == 0 and d > 0:
        # switch orbit_degree_matrix_2 to orbit_degree_matrix_1 ie use the info from graph i and compute the new info for graph i+1
        orbit_degree_matrix_1 = orbit_degree_matrix_2
        node_labels_1 = node_labels_2

        orbit_degree_matrix_2 = np.asarray(read_OrbitDegree_Matrix(next_graph[0]))
        node_labels_2 = np.asarray(read_Node_Labels(next_graph[1]))


    else:

        orbit_degree_matrix_1 = np.asarray(read_OrbitDegree_Matrix(current_graph[0]))
        node_labels_1 = np.asarray(read_Node_Labels(current_graph[1]))  #read_OrbitDegree_Matrices := it reads the orbit degree vectors of all the nodes in a graph

        orbit_degree_matrix_2 = np.asarray(read_OrbitDegree_Matrix(next_graph[0]))
        node_labels_2 = np.asarray(read_Node_Labels(next_graph[1]))


    out = outpath + 'day' + str(current_day) + '_' + 'day' + str(next_day) + '.csv'
    # IF MATRIX, CHANGE HERE. 

    return orbit_degree_matrix_1, orbit_degree_matrix_2, node_labels_1, node_labels_2, out
    

         

    


In [6]:
def read_OrbitDegree_Matrix(file):
    
    ''' Function Purpose: Given an orbit degree matrix file, this function reads it and stores it into a list.'''
    
    orbit_degree_matrix = []

    fRead = open(file, 'r')

    for line in fRead:
        splitted = line.strip().split(' ')
        orbit_degree_matrix.append([int(value) for value in splitted])
    fRead.close()

    return orbit_degree_matrix

In [7]:
def read_Node_Labels(file):
    
    ''' Function Purpose: This functions outputs an array of all the node labels for the rows in the orbit degree matrix'''
    
    node_labels = []

    fRead = open(file, 'r')

    for line in fRead:
        line = line.splitlines()[0].split(' ')
        index = int(line[0])
        node = int(line[1])
        node_labels.append(node - 1) # We subtract one to counter the original offset of +1. Another way to do this is to rerun everything with 'ucsd_txt' instead of 'ucsd_txt_nonzero'
    fRead.close()
    
    return node_labels

# Construct the Orbit Degree Similarity Matrix:

The above set of functions are designed to extract the appropriate files and store the information. Once we have the orbit degree matrices, we want to then perform the similarity measures. This is the purpose of the next set of functions.


In [8]:
def populateSimilarityMatrix(orbit_degree_matrix_1, orbit_degree_matrix_2, SimMatrix):

    ''' Function Purpose: Calculates the similarity between two orbit degree matrices. Stores similarity values in an empty matrix called SimMatrix 
    
        Variables: 
        1. orbit_degree_matrix_1 := the orbit degree matrix for graph 1
        2. orbit_degree_matrix_2 := the orbit degree matrix for graph 2
        3. SimMatrix := an empty matrix for the similarity values.       
        
        Functions Called:
        A. Average_OrbitDistance'''
    
    for node1 in range(0,len(orbit_degree_matrix_1)): #loops over all orbit degree vectors in graph 1
        
        for node2 in range(0,len(orbit_degree_matrix_2)): # loops over all orbit degree vectors in graph 2
            
            
            DGDVS_value = Average_orbitDistance(orbit_degree_matrix_1, orbit_degree_matrix_2, node1, node2)
                # compute the Directed Graphlet Degree Vector Similarity
            
            SimMatrix[node1, node2] = DGDVS_value 
                # store the similarity value in the matrix entry for node 1 and node 2    
                
            ###DiffMatrix[node1, node2] = DGDVS_value
                ### store the DISsimilarity value in the matrix entry for node 1 and node 2
                
    return SimMatrix

## Calculating the Orbit Degree Vector Similarity:

The current similarity metric relies on the arithmetic mean over all the orbit distances.

In [9]:
def Average_orbitDistance(orbit_degree_matrix_1, orbit_degree_matrix_2, node1, node2):
    
    ''' Function Purpose: Computes similarity between to orbit degree vectors based on the average over all orbit distances.
        
        Variables:
        1. orbit_degree_matrix_1 := orbit degree matrix for graph 1
        2. orbit_degree_matrix_2 := orbit degree matrix for graph 2
        3. node1 := Node from orbit degree matrix 1
        4. node2 := Node from orbit degree matrix 2  
        
        Functions Called:
        A. orbitDistance       '''
    
    SumRow = 0
    SumWi = 0
    
    for orbit in range(0, 128):
    # 128 orbits to be considered.
    
        orbit_distance, wi = orbitDistance(orbit_degree_matrix_1, orbit_degree_matrix_2, node1, node2, orbit)
        SumRow = SumRow + orbit_distance
        SumWi = SumWi + wi
    
    Distance = SumRow/SumWi
    GDS_value = 1 - Distance
    # This is the distance between node 1 and node 2 with respect to their graphlet degree vectors
    
    return GDS_value

In [14]:
'''def orbitDistance(orbit_degree_matrix_1, orbit_degree_matrix_2, node1, node2, orbit):
    
        Function Purpose: Computes the difference between two nodes with respect to a single orbit
    
        Variables: 
        1. orbit_degree_matrix_1 := orbit degree matrix for graph 1
        2. orbit_degree_matrix_2 := orbit degree matrix for graph 2
        3. node1 := Node from orbit degree matrix 1
        4. node2 := Node from orbit degree matrix 2 
        5. orbit := The specific orbit to be used.   
        
        Functions Called: 
        A. calcWeight
        B. calcOi        
    
    
    wi = calcWeight(calcOi(orbit))
    
    num = abs(log(orbit_degree_matrix_1[node1, orbit] + 1) - log(orbit_degree_matrix_2[node2, orbit] + 1))
    denom = log(max(orbit_degree_matrix_1[node1, orbit], orbit_degree_matrix_2[node2, orbit]) + 2)

    orbit_distance = wi*num/denom
    
    return orbit_distance, wi '''
    

'def orbitDistance(orbit_degree_matrix_1, orbit_degree_matrix_2, node1, node2, orbit):\n    \n        Function Purpose: Computes the difference between two nodes with respect to a single orbit\n    \n        Variables: \n        1. orbit_degree_matrix_1 := orbit degree matrix for graph 1\n        2. orbit_degree_matrix_2 := orbit degree matrix for graph 2\n        3. node1 := Node from orbit degree matrix 1\n        4. node2 := Node from orbit degree matrix 2 \n        5. orbit := The specific orbit to be used.   \n        \n        Functions Called: \n        A. calcWeight\n        B. calcOi        \n    \n    \n    wi = calcWeight(calcOi(orbit))\n    \n    num = abs(log(orbit_degree_matrix_1[node1, orbit] + 1) - log(orbit_degree_matrix_2[node2, orbit] + 1))\n    denom = log(max(orbit_degree_matrix_1[node1, orbit], orbit_degree_matrix_2[node2, orbit]) + 2)\n\n    orbit_distance = wi*num/denom\n    \n    return orbit_distance, wi '

In [22]:
def orbitDistance(orbit_degree_matrix_1, orbit_degree_matrix_2, node1, node2, orbit):
    
    ''' Function Purpose: Computes the difference between two nodes with respect to a single orbit.
                          Takes into account the absolute difference between orbit degrees. 
    
        Variables: 
        1. orbit_degree_matrix_1 := orbit degree matrix for graph 1
        2. orbit_degree_matrix_2 := orbit degree matrix for graph 2
        3. node1 := Node from orbit degree matrix 1
        4. node2 := Node from orbit degree matrix 2 
        5. orbit := The specific orbit to be used.   
        
        Functions Called: 
        A. calcWeight
        B. calcOi        '''
    
    
    wi = calcWeight(calcOi(orbit))
    
    num = abs(orbit_degree_matrix_1[node1, orbit] - orbit_degree_matrix_2[node2, orbit])
    denom = max(orbit_degree_matrix_1[node1, orbit], orbit_degree_matrix_2[node2, orbit])
    
    if denom == 0:
        orbit_distance = 0
    else:
        orbit_distance = wi*num/denom
    
    return orbit_distance, wi

In [11]:
def calcWeight(oi):
    wi = 1 - (log(oi))/(log(129))
    return wi

In [12]:
def calcOi(orbit):
    
    ''' Function Purpose: Receives an orbit number and outputs the number of other orbits in which it depends on. 
        Ex: A center node (/orbit) in a star depends on the number of times the node is a source node in a directed edge.'''
    oi_1 = [0, 1]
    oi_2 = [2,4,5,6,7,8,10,11]
    oi_3 = [3,9,12,13,16,17,20,21,24,25,28,29,30,31,32,33,36,60,64,68,72,118,120,125,128]
    oi_4 = [19,22,23,27,35,38,44,45,47,48,52,56,57,61,63,66,67,70,73,74,76,77,78,80,85,87,91,92,95,96,100,105,113,115]
    oi_5 = [14,15,18,26,34,37,42,43,49,50,53,54,58,59,62,65,69,71,88,90,93,94,99,103,107,111,121,122,126,127]
    oi_6 = [39,40,41,46,51,55,75,79,81,82,83,84,86,89,97,98,101,102,104,106,108,109,110,112,114,116,117,119,123,124]
    if orbit in oi_1:
        return 1
    elif orbit in oi_2:
        return 2
    elif orbit in oi_3:
        return 3
    elif orbit in oi_4:
        return 4
    elif orbit in oi_5:
        return 5
    elif orbit in oi_6:
        return 6
    else:
        return 0

# From Similarity Matrix to Similarity List:

We should have a simialrity matrix whose entries are the orbit degree vector similarity value (ie the average orbit difference). We might want the output to be in a list format rather than matrix. This is the next function.

In [13]:
def convertSimMatrix2SimList(matrix):
    
    ''' Function Purpose: Converts the similarity matrix pandas dataframe and converts it to a list of similarities
    where the first column is the node 1 from graph1, second column is the node 2 from graph2, and the third
    column is the similarity value. The order goes from all the pairings with node1 to all the other nodes in graph 2.'''
    
    sim_matrix = matrix
    
    node_pair_similarities = []
    
    graph1_node_array = list(sim_matrix.index)
    
    graph2_node_array = list(sim_matrix.columns)

    for i in range(0, len(graph1_node_array)): # index through the vertices of graph 1, (rows)

        node1 = graph1_node_array[i]

        for j in range(0, len(graph2_node_array)): # index through the vertices of graph 2, (columns)

            node2 = graph2_node_array[j]

            sim_value = sim_matrix.loc[node1, node2] # extract the entry value in (row, column)

            node_pair_similarities.append([node1, node2, sim_value]) # append to the list of nodes and their similarities
            
    return node_pair_similarities
            

# MAIN CODE:

This is the main code. 

## Options:
The main code has options:

1. If taking time graphs separated by one or more time stamps. If '0', then time graphs are adjacent:
    GAP >= 0 

2. Similarity matrix output format:
    OUTPUT_FORMAT in { 'matrix' , 'list' }
    
3. Timing how long the creation of the similarity matrix or similarity list takes?:
    STOP_WATCH in { 'True' , 'False' }

## Comments:
The main code has two kinds of comments:

1. ' # ' The single pound sign is a comment of the specific line of code.

2. ' ## ' The double pound sign is a comment about the succeeding code's role within the organization of the code.
    
    




## Making Directories for gaps greater than 0

We will make a directory if we have gap greater than 0. This is included when determing the outpath of the matrices.

In [16]:
def In_Out_Paths(GAP, OUTPUT_FORMAT, cwd):
    
    ''' Function Purpose: Picks the correct inpath and outpaths. The out paths considers if user wants gaps in days or not. 
    Ie to consider consecutive time graphs or some other extension of time. Also, considers if user wants lists or not.
    
    Varaibles:
    1. GAP := How far apart are the time graphs' time points?
    2. OUTPUT_FORMAT := Do you want it in a list format?
    3. cwd := current working directory '''
    
    inpath = None
    outpath = None
    outpath2 = None
    
    ## SET INPATHS & OUTPATHS 
    # Outpaths differentiates y/n gaps, and y/n lists
        
    ## Inpath:
    inpath = cwd + '/wells/well' + well_number + '/'  # CHANGE HERE

    ## Outpath:
    if GAP == 0:
        GDS_dir = '/GDS_days/'

    ## Outpath_gaps:
    elif GAP > 0:
        parent_dir = '/GDS_gap_days/'

        gap_dir = 'GDS_gap_' + str(GAP) + '_days/'
        
        GDS_dir = parent_dir + gap_dir                    # Specifies the gap in the directory
        
    ## Outpath needs gaps or not:
    outpath = cwd + GDS_dir + 'well' + well_number + '/'  # where to place the similarity matrices

    ## Asks if directory exists:
    if os.path.isdir(outpath) == False:
        os.makedirs(outpath)
        print("Directory '% s' created" % outpath)

    ## Outpath_lists:
    if OUTPUT_FORMAT == 'list':

        outpath2 = cwd + GDS_dir + 'well' + well_number + '_lists/' # where to place the similarity lists

        if os.path.isdir(outpath2) == False:
            os.makedirs(outpath2)
            print("Directory '% s' created" % outpath2)
    
    
    return inpath, outpath, outpath2
    
    

In [ ]:
WELLS = ['22', '23', '34']

## SET OPTIONS ------------
for GAP in range(0, 4):
    
    #GAP = 4  # CHANGE # Nonnegative number indicating the difference between time of compared graphs. '0' indicates adjacent time graphs.

    OUTPUT_FORMAT = 'list'   # If 'list', then does the procedure to convert matrix to list. Right now, 'list' produces both matrix and list format.

    STOP_WATCH = False       # If 'True', then does timer operations



    for w in WELLS:

        well_number = w # CHANGE


        ## SET INPATHS & OUTPATHS 

        # Paths:
        inpath, outpath, outpath2 = In_Out_Paths(GAP, OUTPUT_FORMAT, cwd)



        ## INITIALIZE PARAMETERS, ARRAYS, & OPTIONS

        ## Parameters:
        gap = GAP

        ## Arrays:
        orbit_degree_matrix_1 = []
        orbit_degree_matrix_2 = []

        node_labels_1 = []
        node_labels_2 = []

        ## Options:
        if OUTPUT_FORMAT == 'list':
            Similarity_Matrices_Set = []

        if STOP_WATCH == True:
            array_time_for_matrix_df = []
            array_time_for_list_df = []
            array_time_for_matrix2list_conversion = []



        ## COMPUTING THE SIMILARITY BETWEEN TIME GRAPHS ACCORDING TO THEIR ORBIT DEGREE MATRICES

        ## Get all the orbit degree matrices for the well (from txt to dictionary):
        orbit_degree_matrices_dictionary, days = make_Orbit_Degree_Matrix_Dictionary(inpath)


        for d in range(0,len(days)-1): 

            if (d + gap + 1) < len(days):     # This ensures we do not index out of range.

                # Set days.
                current_day = days[d]
                next_day = days[d + gap + 1] # 'gap' = 0 refers to adjacent time points.

                ## Start the timer:
                if STOP_WATCH == True:
                    start = time.time()

                ## Get the orbit degree matrices (from dictionary to arrays):
                orbit_degree_matrix_1, orbit_degree_matrix_2, node_labels_1, node_labels_2, out = store_Data_For_TimeGraphs(orbit_degree_matrices_dictionary, current_day, next_day, d, gap, orbit_degree_matrix_1, orbit_degree_matrix_2, node_labels_1, node_labels_2)


                #M = max(len(SigFile1), len(SigFile2)) # M := maximum node set size between the two graphs.      

                # DiffMatrix = np.ones((M , M))  
                # DiffMatrix has dimensions equal to the maximum node set. Fill entries with '1' since '1' indicates maximum distance/difference. This is mostly related to unaligned nodes. 
                # SimMatrix = np.zeros((M, M))

                ## Compute the matrix with similarity values:
                SimMatrix = np.zeros((len(orbit_degree_matrix_1), len(orbit_degree_matrix_2)))

                SimMatrix = populateSimilarityMatrix(orbit_degree_matrix_1, orbit_degree_matrix_2, SimMatrix)
                print(SimMatrix[0])

                df = pd.DataFrame(data=SimMatrix, index=node_labels_1, columns=node_labels_2)

                df.to_csv(out)


                ## Get time to build matrix:
                if STOP_WATCH == True:

                    stop1 = time.time()



                ## Get similarity lists:
                if OUTPUT_FORMAT == 'list':

                    #Similarity_Matrices_Set.append(df)

                    out2 = outpath2 + 'day' + str(current_day) + '_' + 'day' + str(next_day) + '.csv' # CHANGE 

                    node_pair_similarities = convertSimMatrix2SimList(df) 

                    df_2 = pd.DataFrame(data=node_pair_similarities, index=None, columns=['node 1', 'node 2', 'similarity'])

                    df_2.to_csv(out2)


                ## Get times:
                if STOP_WATCH == True:

                    stop2 = time.time()

                    time_for_matrix_df = stop1 - start

                    if OUTPUT_FORMAT == 'list':

                        time_for_list_df = stop2 - start

                        time_for_matrix2list_conversion = stop2 - stop1

                    #array_time_for_matrix_df.append(time_for_matrix_df)
                    #array_time_for_list_df.append(time_for_list_df)
                    #array_time_for_matrix2list_conversion.append(time_for_matrix2list_conversion)

                    print('time_for_matrix_df: ',time_for_matrix_df, '\n', 'time_for_list_df: ', time_for_list_df, '\n', 'time_for_matrix2list_conversion', time_for_matrix2list_conversion)







                

[0.6885196  0.7652016  0.49825551 0.20857223 0.66919616 0.19214212
 0.82397027 0.73005418 0.83444427 0.77397322 0.74696695 0.46031637
 0.58330853 0.72335384 0.65427267 0.95258701 0.12771835 0.11964986
 0.69259034 0.70584383 0.68199937 0.75949501 0.77349197 0.60656986
 0.78989818 0.74496663 0.47141131 0.63216271 0.38423946 0.71598294
 0.66535994 0.76302982 0.74876441 0.68431962 0.76496267 0.70810603
 0.54978609 0.27249992 0.90197774 0.829075   0.71000141 0.94112186
 0.74606168 0.72385986 0.83215939 0.8214471  0.19685448 0.72743625
 0.63040177 0.73395226 0.90321001 0.73952036 0.65717708 0.65074336
 0.67644068 0.90093356 0.75589083 0.93320258 0.56805584 0.69029397
 0.75643337 0.75643337 0.67004615]
[0.64811991 0.64606396 0.14855396 0.60455225 0.6775371  0.65574024
 0.69885471 0.72358861 0.66384424 0.48185988 0.78664309 0.22833274
 0.6775371  0.67160508 0.72358861 0.66036886 0.68852142 0.23947638
 0.72358861 0.66115886 0.66051086 0.59210244 0.6038247  0.46877022
 0.72049746 0.28104547 0.69

[0.79692655 0.1515269  0.50837738 0.62425689 0.07592473 0.09122078
 0.73239012 0.67808273 0.61663968 0.7058343  0.2142193  0.10413015
 0.67419777 0.75156771 0.69697634 0.14075161 0.1722524  0.70931037
 0.6896114  0.59688178 0.72927358 0.16549924 0.67807965 0.68938109
 0.77723447 0.25502555 0.77869674 0.79692655 0.67807965 0.81529839
 0.76331288 0.52986927 0.77723447 0.77045856 0.56179218 0.61501754
 0.61019855 0.67807965 0.74259376 0.65307771 0.67335115 0.68777854
 0.68777854 0.70320778 0.71506422 0.73748993 0.83718852 0.6610096
 0.67807965 0.83718852 0.60391713 0.67808273 0.67808273 0.4112863
 0.62178149 0.68777151 0.6644338  0.71557302 0.67808273 0.73911793
 0.64152896 0.71557302 0.52392464 0.73325744 0.46591154 0.67807965
 0.71557302 0.78727365 0.78282471 0.64842209 0.69394013 0.66498185
 0.77045856 0.67419777 0.6950792  0.67846705 0.61263481 0.74184664
 0.58419776 0.42567973 0.78601814 0.6589382  0.71852878 0.78601814
 0.78601814 0.70078328 0.81529839 0.61664464 0.61664464 0.678082

[0.41952833 0.50292364 0.45752739 0.44082101 0.48267496 0.51211649
 0.34615179 0.47491337 0.47892491 0.44564538 0.27851707 0.47347729
 0.45832324 0.60494498 0.20191565 0.52466154 0.46350923 0.44107751
 0.49370741 0.4833136  0.44564125 0.4176406  0.53806616 0.45185023
 0.49983766 0.43394687 0.40785998 0.52532814 0.51569948 0.42990218
 0.44389719 0.61664934 0.50385682 0.44173195 0.46009436 0.51547477
 0.40500931 0.47186609 0.59185264 0.44389719 0.49554973 0.59286095
 0.5289601  0.61366111 0.46836675 0.52783333 0.47706666 0.59286095
 0.49744587 0.5667695  0.49922606 0.53686395 0.59286095 0.51669791
 0.50003147 0.51513909 0.49288636 0.4082799  0.45185023 0.46836675
 0.4646758  0.54273514 0.59286095 0.5030904  0.48071239 0.48345082
 0.47065666 0.44206469 0.50933649 0.39275699 0.52575169 0.52871502
 0.45623343 0.59286095 0.3986255  0.60664294 0.51814663 0.44291469
 0.48057132 0.44174591 0.44288343 0.46836675 0.62983559 0.59286095
 0.54280865 0.54575046 0.53252892 0.47323865 0.59259224 0.4532

[0.5937674  0.57246057 0.21888171 0.78291095 0.47993025 0.75029
 0.44631476 0.75253487 0.72170614 0.68216883 0.21001171 0.89823898
 0.76583187 0.71731172 0.66881059 0.6375068  0.89823898 0.42303299
 0.74994207 0.46903288 0.69434018 0.424644   0.21257205 0.72284465
 0.81505663 0.75790371 0.82758286 0.89823898 0.7833492  0.82015683
 0.75349183 0.66143807 0.75828064 0.72393421 0.70123392 0.89823898
 0.76242424 0.73010979 0.76311157 0.4761667  0.89823898 0.6245397
 0.21282265 0.89823898 0.51936704 0.71812661 0.89823898 0.69774372
 0.81471003 0.71812661 0.89823898 0.52315748 0.73665194 0.74977325
 0.62611703 0.89823898 0.68592954 0.82127791 0.78899036 0.79167268
 0.58543265 0.78899036 0.89823898 0.89823898 0.68478578 0.68216115
 0.68396377 0.75582155 0.62400611 0.72437525 0.81473625 0.59003559
 0.89823898 0.7810808  0.79715738 0.7444475  0.75002686 0.76314987
 0.54165283 0.66628913 0.71646364 0.81539886 0.82015683 0.77365032
 0.71547217 0.65495437 0.74444344 0.71812661 0.79292596 0.78333547

[0.94198224 0.45138965 0.94198224 0.94198224 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.86294095 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224
 0.84899681 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.86294095 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.85024756 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224
 0.94198224 0.94198224 0.94198224 0.94198224 0.94198224 0.9419

...

[0.72257219 0.69741266 0.67942684 0.69208687 0.20585784 0.23606724
 0.63214084 0.56772271 0.63327616 0.61908656 0.74780868 0.73866811
 0.71838082 0.47731449 0.59510332 0.41962667 0.66649346 0.59507551
 0.70239114 0.74780868 0.5510205  0.74780868 0.63518891 0.63518891
 0.63518891 0.73866811 0.56284323 0.74780868 0.68321929 0.56772271
 0.51106509 0.47623377 0.61908656 0.63336641 0.74780868 0.6449401
 0.71838082 0.68321929 0.80723836 0.65920065 0.58341846 0.64907021
 0.78714638 0.23177575 0.61514541 0.61852247 0.61549863 0.66836449
 0.74399211 0.74399211 0.66746035 0.67702152 0.55322551 0.65302834
 0.65243494 0.65138016 0.66089061 0.72008746 0.69992196 0.63755069
 0.68269253 0.6160935  0.63558486 0.66746035 0.78714638 0.78714638
 0.54857189 0.6890663  0.63342884 0.56430395 0.65164516 0.52396346
 0.78714638 0.70998546 0.72615706 0.72234345 0.78714638 0.78714638
 0.78714638 0.78714638 0.64091924 0.66044064 0.72234345 0.78714638
 0.72234345 0.78714638 0.78714638 0.62772134 0.64861711 0.72923

[0.56980148 0.21086351 0.74633711 0.7476274  0.78449561 0.66743884
 0.71913165 0.73202684 0.75339807 0.68646861 0.74614488 0.69435665
 0.7476274  0.73120778 0.77521562 0.68928372 0.78489165 0.24770466
 0.53972627 0.66898281 0.67139395 0.53973147 0.67139395 0.5933917
 0.67139395 0.67139395 0.75316551 0.71148552 0.67139395 0.70581784
 0.67139395 0.67139395 0.75316551 0.67139395 0.6864849  0.67139395
 0.74564969 0.67251907 0.75316551 0.81840768 0.79774675 0.89763189
 0.39247629 0.74672169 0.69226536 0.69226536 0.73691208 0.6763251
 0.72267448 0.77767968 0.75547488 0.74672169 0.73691208 0.6157975
 0.76248157 0.74672169 0.60986472 0.77767968 0.72267448 0.7923181
 0.89763189 0.89763189 0.72818513 0.72860723 0.66039049 0.89763189
 0.89763189 0.89763189 0.89763189 0.89763189 0.7923181  0.89763189
 0.7923181  0.66912319 0.89763189 0.89763189 0.76017064 0.89763189
 0.89763189 0.81401504 0.89763189 0.85593406 0.35266748 0.83511693
 0.66039049 0.89763189 0.7923181  0.77401711 0.68052261 0.86346259

[0.14828384 0.358119   0.08043271 0.052933   0.12212626 0.06679781
 0.13570537 0.34358332 0.05396356 0.11470499 0.12085087 0.07825656
 0.10770144 0.14654036 0.14830465 0.13233703 0.12130909 0.17380764
 0.11733067 0.15173368 0.24027639 0.22348144 0.30977965 0.13139961
 0.19990461 0.05976402 0.13358135 0.13401436 0.2087729  0.27280291
 0.13477102 0.08446984 0.20892986 0.07778541 0.13403618 0.12980556
 0.14250329 0.07075463 0.08408082 0.21672893 0.09116757 0.13173847
 0.08892952 0.19233759 0.16051004 0.09767798 0.12574066 0.07570941
 0.07738914 0.06310628 0.0546504  0.03073797 0.08757438 0.00724197
 0.14689111 0.04734459 0.05362635 0.05497861 0.13682669 0.03561024
 0.02548278]
[0.58482447 0.23227755 0.58482447 0.58482447 0.58482447 0.62471966
 0.58482447 0.47443619 0.54605736 0.54605736 0.58482447 0.62947388
 0.58482447 0.47290082 0.47443619 0.56335633 0.47334143 0.58482447
 0.58482447 0.47443619 0.54605736 0.58482447 0.58482447 0.58482447
 0.58482447 0.54605736 0.46823623 0.40556729 0.30

[0.70968056 0.33705562 0.70968056 0.70968056 0.70968056 0.70968056
 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056
 0.70968056 0.70968056 0.6157093  0.62489316 0.70968056 0.70968056
 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056 0.63235592
 0.66443979 0.70968056 0.53382279 0.70968056 0.70968056 0.67893794
 0.70968056 0.70968056 0.66277598 0.74724111 0.70968056 0.70968056
 0.70968056 0.56382691 0.49659445 0.70968056 0.70968056 0.6157093
 0.62489316 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056
 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056
 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056
 0.70968056 0.70968056 0.70968056 0.57301086 0.6157093  0.62489316
 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056
 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056
 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056
 0.70968056 0.70968056 0.70968056 0.70968056 0.70968056 0.37534

[0.56148083 0.7417439  0.13553707 0.75103107 0.45235239 0.53646127
 0.19248609 0.68521209 0.59260209 0.65318474 0.66033982 0.61671592
 0.68006427 0.68006427 0.59959577 0.5867983  0.59487813 0.5697564
 0.52380427 0.62135875 0.64100999 0.52806571 0.53248134 0.59134294
 0.5551163  0.2759427  0.59755437 0.58019754 0.60134838 0.50647149
 0.49688262 0.57210408 0.53588861 0.5694845  0.59154405 0.66100847
 0.58019754 0.68006427 0.52354733 0.55684547 0.65472574 0.58415344
 0.6136295  0.44836935 0.58672218 0.5715055  0.5715055  0.57476581
 0.5671067  0.56065383 0.58513192 0.57095211 0.65472574 0.62846684
 0.50840113 0.52217594 0.64989209 0.68006427 0.49230989 0.591103
 0.68006427 0.56582918 0.65527327 0.47049593 0.63641279 0.50964731
 0.68006427 0.52184157 0.57361218 0.52216821 0.79676678 0.54451079
 0.76132023 0.52549131 0.68006427 0.5715055  0.56213566 0.58019754
 0.60813375 0.52009647 0.65472574 0.58019754 0.68006427 0.68006427
 0.66014615 0.62586232 0.52969909 0.51599181 0.81303318 0.5830085

In [ ]:
node_labels_dictionary = None
    # (str) File path to get the dictionary. 

class OrbitDegreeSimilarityGraph:
    ''' 
    This takes the orbit degree matrices of two graphs and 
    compares them via their nodes' orbit degree vectors similarities.

    '''
    number_of_orbits = 129
    # (int) Total number of orbits
    
    
    
    def __init__(self, orbit_degree_matrix_1, orbit_degree_matrix_2, node_labels_dictionary):  
        
        self.orbit_degree_matrix_1 = orbit_degree_matrix_1
        self.orbit_degree_matrix_2 = orbit_degree_matrix_2
        self.orbits = range(0, number_of_orbits)
        self.similarity_graph = nx.complete_bipartite_graph(len(self.orbit_degree_matrix_1), (len(self.orbit_degree_matrix_2)))
        self.node_labels_for_graph_1 = 
        # label the nodes.
        # get mapping. (dictionary to convert node labels)
        
        self.similarity_graph = nx.relabel_nodes(self.similarity_graph, node_labels, copy=False)
        
        
        
        self.similarity_list = np.zeros((len(self.orbit_degree_matrix_1) * len(self.orbit_degree_matrix_2)), 2)
        self.SimMatrix = np.zeros((len(self.orbit_degree_matrix_1), len(self.orbit_degree_matrix_2)))
        
    
    def makeSimilarityGraph(self):
        
        for v1 in range(0,len(self.orbit_degree_matrix_1)): 
        #loops over all orbit degree vectors in graph 1

            for v2 in range(0,len(self.orbit_degree_matrix_2)): 
            # loops over all orbit degree vectors in graph 2
                
                DGDVS_value = Average_orbitDistance(self.orbit_degree_matrix_1, self.orbit_degree_matrix_2, v1, v2)
                # compute the Directed Graphlet Degree Vector Similarity
                
                self.similarity_graph[v1][v2]['weight'] = DGDVS_value

    
    
    def populateSimilarityMatrix(self): 

    ''' Function Purpose: Calculates the similarity between two orbit degree matrices. Stores similarity values in an empty matrix called SimMatrix 
    
        Variables: 
        1. self.orbit_degree_matrix_1 := the orbit degree matrix for graph 1
        2. self.orbit_degree_matrix_2 := the orbit degree matrix for graph 2
        3. self.SimMatrix := an empty matrix for the similarity values.       
        
        Functions Called:
        A. Average_OrbitDistance'''

        for node1 in range(0,len(self.orbit_degree_matrix_1)): 
        #loops over all orbit degree vectors in graph 1

            for node2 in range(0,len(self.orbit_degree_matrix_2)): 
            # loops over all orbit degree vectors in graph 2

                DGDVS_value = Average_orbitDistance(self.orbit_degree_matrix_1, self.orbit_degree_matrix_2, node1, node2)
                # compute the Directed Graphlet Degree Vector Similarity

                self.SimMatrix[node1, node2] = DGDVS_value 
                # store the similarity value in the matrix entry for node 1 and node 2    

        
        
    def Average_orbitDistance(self, node1, node2):
    
    ''' Function Purpose: Computes similarity between to orbit degree vectors based on the average over all orbit distances.
        
        Variables:
        1. orbit_degree_matrix_1 := orbit degree matrix for graph 1
        2. orbit_degree_matrix_2 := orbit degree matrix for graph 2
        3. node1 := Node from orbit degree matrix 1
        4. node2 := Node from orbit degree matrix 2  
        
        Functions Called:
        A. orbitDistance       '''

        SumRow = 0
        SumWi = 0

        for orbit in self.orbits:

            orbit_distance, wi = self.orbitDistance(self.orbit_degree_matrix_1, self.orbit_degree_matrix_2, node1, node2, orbit)
            SumRow = SumRow + orbit_distance
            SumWi = SumWi + wi

        Distance = SumRow/SumWi
        GDS_value = 1 - Distance
        # This is the distance between node 1 and node 2 with respect to their graphlet degree vectors

        return GDS_value
        
        
    def orbitDistance(self, node1, node2, orbit):
    
    ''' Function Purpose: Computes the difference between two nodes with respect to a single orbit
    
        Variables: 
        1. orbit_degree_matrix_1 := orbit degree matrix for graph 1
        2. orbit_degree_matrix_2 := orbit degree matrix for graph 2
        3. node1 := Node from orbit degree matrix 1
        4. node2 := Node from orbit degree matrix 2 
        5. orbit := The specific orbit to be used.   
        
        Functions Called: 
        A. calcWeight
        B. calcOi        '''


        wi = self.calcWeight(self.calcOi(orbit))

        num = abs(log(orbit_degree_matrix_1[node1, orbit] + 1) - log(orbit_degree_matrix_2[node2, orbit] + 1))
        denom = log(max(orbit_degree_matrix_1[node1, orbit], orbit_degree_matrix_2[node2, orbit]) + 2)

        orbit_distance = wi*num/denom

        return orbit_distance, wi
 
    
    def calcOi(orbit):
    
    ''' Function Purpose: Receives an orbit number and outputs the number of other orbits in which it depends on. 
        Ex: A center node (orbit) in a star depends on the number of times the node is a source node in a directed edge.'''
        
        oi_1 = [0, 1]
        oi_2 = [2,4,5,6,7,8,10,11]
        oi_3 = [3,9,12,13,16,17,20,21,24,25,28,29,30,31,32,33,36,60,64,68,72,118,120,125,128]
        oi_4 = [19,22,23,27,35,38,44,45,47,48,52,56,57,61,63,66,67,70,73,74,76,77,78,80,85,87,91,92,95,96,100,105,113,115]
        oi_5 = [14,15,18,26,34,37,42,43,49,50,53,54,58,59,62,65,69,71,88,90,93,94,99,103,107,111,121,122,126,127]
        oi_6 = [39,40,41,46,51,55,75,79,81,82,83,84,86,89,97,98,101,102,104,106,108,109,110,112,114,116,117,119,123,124]
        
        if orbit in oi_1:
            return 1
        elif orbit in oi_2:
            return 2
        elif orbit in oi_3:
            return 3
        elif orbit in oi_4:
            return 4
        elif orbit in oi_5:
            return 5
        elif orbit in oi_6:
            return 6
        else:
            return 0
        
       
    def calcWeight(orbit):
        
        ''' Function Purpose: Computes the orbit weight.'''
        
        wi = 1 - (log(orbit))/(log(number_of_orbits))
        return wi
        
    
    
        